In [11]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable

In [26]:
class App:

    def __init__(self, uri, user, password):
        self.user = user
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()
    
    @staticmethod
    def _find_skill(tx, skill_name):
        query = (
            "MATCH (s:Skill) "
            "WHERE toLower(s.name) = toLower($SKILL_NAME) "
            "RETURN s.name AS skill_name"
        )
        result = tx.run(query, SKILL_NAME=skill_name)
        return [row["skill_name"] for row in result]

    def find_skill(self, skill_name):
        with self.driver.session(database=self.user) as session:
            result = session.execute_read(self._find_skill, skill_name)
            if len(result) == 0:
                print("No results found")
                return False
            else:
                for row in result:
                    print("Found skill: {row}".format(row=row))
                return True
    
    @staticmethod
    def _create_skill(tx, skill_name):
        query = (
            "MERGE (s:Skill { name: $SKILL_NAME })"
        )
        result = tx.run(query, SKILL_NAME=skill_name)
        # print(result)

    @staticmethod
    def _create_skill_relation(tx, skill0_name, skill1_name):
        query = (
            "MATCH (s0: Skill), (s1: Skill) "
            "WHERE toLower(s0.name) = toLower($SKILL0) AND toLower(s1.name) = toLower($SKILL1) "
            "AND s0.attr is NULL AND s1.attr is NULL "
            "MERGE  (s0)-[:CONTAINS_SKILL]->(s1) "
            "RETURN s0, s1"
        )
        result = tx.run(query, SKILL0=skill0_name, SKILL1=skill1_name)
        try:
            return [{"s0": row["s0"]["name"], "s1": row["s1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise
    
    def addToSkillTree(self, parentSkill: str, childSkill: str):

        with self.driver.session(database=self.user) as session:
            session.execute_write(self._create_skill, parentSkill)
            session.execute_write(self._create_skill, childSkill)
            session.execute_write(self._create_skill_relation, parentSkill, childSkill)

    def create_skill_tree(self, skill_json: dict, parentSkill=None):
        for currentSkill, value in skill_json.items():
        # print the current key-value pair with appropriate indentation
            if parentSkill is not None:
                print(parentSkill, currentSkill)
                # add the current key-value pair to the skill tree
                self.addToSkillTree(parentSkill, childSkill=currentSkill)
                
            if isinstance(value, dict):
                # if the value is a nested dictionary, recursively print it out with increased indentation
                # print('')
                self.create_skill_tree(value, parentSkill=currentSkill)
            else:
                # otherwise, print the value
                print(value)





In [27]:
uri = "neo4j+s://f9f049f6.databases.neo4j.io"
user = "neo4j"
password = "MPPvraZHeQtIOymDIYIGlINLjNq_fn6iswgM6GY2HAQ"
app = App(uri, user, password)

In [ ]:
app.find_skill("JavaScript")

In [24]:
import json

with open('skillTree.json', 'r') as skillTreeFile:
    data = json.load(skillTreeFile)

In [ ]:
def print_json(data, parentKey):

    for key, value in data.items():
        # print the current key-value pair with appropriate indentation
        if len(parentKey) != 0:
            print(parentKey, key)
            
        if isinstance(value, dict):
            # if the value is a nested dictionary, recursively print it out with increased indentation
            # print('')
            print_json(value, parentKey=key)
        else:
            # otherwise, print the value
            print(value)

# call the function on the loaded JSON data
print_json(data, parentKey='')

In [ ]:
app.create_skill_tree(skill_json=data, parentSkill=None)
app.close()